In [57]:
import re
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [58]:
site_data = {}

In [59]:
def get_categories():
    categories = []
    
    url = 'http://www.shamela.ws/index.php/categories'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "lxml")

    for category in soup.find_all('li', {'class':'regular-cat'}):
        cat_name = str(category.find('a').text)
        cat_link = str(category.find('a').attrs['href'])
        if re.match('/index\.php/category/\d+',cat_link):
            categories.append((cat_name, cat_link))
            #print(cat_name, cat_link)
    
    with open('categories.json', 'w') as fp:
        json.dump(categories, fp)
    
    return categories

In [60]:
def get_last_category_page(soup):
    try: 
        last_page = soup.find_all('a', {'href': re.compile(r'/index\.php/category/\d+/page-\d+')})[1]
        return re.findall(r'/index\.php/category/\d+/page-(\d+)',str(last_page))[0]
    except:
        return '1'

In [65]:
def get_books():
    categories = get_categories()
    
    for category in categories[:-1]:
        url = 'http://www.shamela.ws'+category[1]
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "lxml")
        last_page = get_last_category_page(soup)
        
        books = []
        
        for page in range(1,int(last_page)+1):
            url = 'http://www.shamela.ws'+category[1]+'/page-'+str(page)
            response = requests.get(url)
            soup = BeautifulSoup(response.text, "lxml")
            
            print('Category: ', category[0], ' ', category[1],'Page No. ',page)
            for book in soup.find_all('td', {'class':'regular-book'}):
                try:
                    book_name = str(book.find('a').text)
                    book_link = str(book.find('a').attrs['href'])
                    if re.match('/index\.php/book/\d+',book_link):
                        #print(book_name,book_link)
                        books.append((book_name,book_link))
                except:
                    print('Except: ', book)
        
        site_data[str(category[1])] = books
        
    with open('cats_books.json', 'w') as fp:
        json.dump(site_data, fp)

In [70]:
index_url = 'index.php/book/124336'

In [69]:
re.findall

'http://www.shamela.ws/index.php/book/124336'

In [ ]:
def get_book_url(index_url):
    book_url = 'http://shamela.ws/browse.php/book-'